In [12]:
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from keras.layers import Dense, Flatten
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
import random as rn
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout
import matplotlib.pyplot as plt
from keras.regularizers import l1, l2
from tensorflow.keras.layers import LeakyReLU
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from PIL import Image


from tensorflow import keras
from keras_tuner import RandomSearch

In [2]:
img_width = 320
img_height = 320

X = []
y = []

In [3]:
daisy_path = "flowers/daisy"
dandelion_path = "flowers/dandelion"
rose_path = "flowers/rose"
sunflower_path = "flowers/sunflower"
tulip_path = "flowers/tulip"


In [4]:
def make_training_data(flower_type,folder_path):
    # Iterate over the images in the directory
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
            image_path = os.path.join(folder_path, filename)
            
            # Open and resize the image
            image = Image.open(image_path)
            image = image.resize((img_width, img_height))
            
            # Convert the image to a numpy array and store it in the list
            X.append(np.array(image))
            y.append(str(flower_type))

In [5]:
make_training_data('Daisy',daisy_path)
print(len(X))

764


In [6]:
make_training_data('Dandelion',dandelion_path)
print(len(X))

make_training_data('Rose',rose_path)
print(len(X))

make_training_data('Sunflower',sunflower_path)
print(len(X))

make_training_data('Tulip',tulip_path)
print(len(X))

1816
2600
3333
4317


In [7]:
# image preprocessing
le=LabelEncoder()
Y=le.fit_transform(y)
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255

In [8]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)


In [13]:
# creating model with hyperparameter tuner

# Define the model-building function
def build_model(hp):
    model = Sequential()

    # First convolution layer
    model.add(Conv2D(filters=hp.Int('filters_1', min_value=16, max_value=64, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     strides=(1, 1),
                     input_shape=(320, 320, 3)))
    model.add(Activation('relu'))

    # Additional convolution layers
    for i in range(hp.Int('conv_layers', min_value=1, max_value=3)):
        model.add(Conv2D(filters=hp.Int(f'filters_{i+2}', min_value=16, max_value=64, step=16),
                         kernel_size=(3, 3),
                         padding='same',
                         strides=(1, 1)))
        model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    # MLP
    model.add(Flatten())
    model.add(Dense(hp.Int('dense_units', min_value=256, max_value=1024, step=256)))
    model.add(Activation('relu'))
    model.add(Dense(5, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [14]:

#feeding the model and parameters to Random Search
tuner = RandomSearch(build_model,
                    objective = 'val_accuracy',
                    max_trials = 4,
                    executions_per_trial = 3,
                    directory = 'tuner1',
                    project_name = 'flower identification')

# So this will run for (4*3)= 12 times with 10 epoch

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-07-14 20:14:42.298200: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-14 20:14:42.298933: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
# This tells us how many hyperparameter we are tuning

tuner.search_space_summary()

Search space summary
Default search space size: 4
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
conv_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
filters_2 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 256, 'max_value': 1024, 'step': 256, 'sampling': 'linear'}


In [16]:
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Trial 3 Complete [01h 09m 56s]
val_accuracy: 0.4898147980372111

Best val_accuracy So Far: 0.5830246806144714
Total elapsed time: 02h 16m 50s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
64                |16                |filters_1
3                 |1                 |conv_layers
64                |16                |filters_2
1024              |512               |dense_units
16                |32                |filters_3
64                |48                |filters_4



: 

: 

In [1]:
# We can see the hyperparameters which is been selected by KerasTuner

tuner.results_summary()

NameError: name 'tuner' is not defined